In [13]:
#Carregando as bibliotecas necessárias
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [14]:
#Importando dataset
data = keras.datasets.imdb

In [15]:
#Splittando entre textos de teste e textos de treino
#O num words é para ele selecionar as palavras que ocorrem mais de 10000 vezes no dataset
(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words=10000)

#Você vai obter um vetor enorme de números
#Cada número se refere a uma palavra, como um dicionário
f"{train_data[0]}"



'[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]'

In [16]:
#Essa função eu não entendi direito, mas ele, resumidamente tá pegando o vetor cheio de número (integer encoded) e traduzindo para texto
#Aqui ele tá pegando o dicionário pra "traduzir" os números do vetor
_word_index = keras.datasets.imdb.get_word_index()

word_index = {k:(v+3) for k,v in _word_index.items()}
#Isso é o que iremos inserir caso não houver caracter nenhum no texto
word_index["<PAD>"] = 0
#Esse parâmetro fala aonde devemos começar a ler no texto
word_index["<START>"] = 1
#Caso tenha caracteres desconhecidos, ele irá printar UNK
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

#Essa vai ser a função chamada para decodificar o integer do predict
def decode_review(text):
	return " ".join([reverse_word_index.get(i, "?") for i in text])

In [17]:
#Nesse dataset, temos texto de diferentes tamanhos, o que é inadimissível para a neural network, já que ela "precisa" ter um tamanho fixo, logo, vamos cortar ou preencher os textos que não possuem esse tamanho
#Caso o texto seja menor: será preenchido. Caso o texto seja maior: será cortado
#1º parâmetro - Qual o data que queremos alterar
#value - O que queremos colocar, caso o texto seja menor que 250
#3º parâmetro - Decidir cortar o que vem depois do 250º caracter, ou antes
#maxlen - número de caracteres que desejamos na saída
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], padding="post", maxlen=250)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], padding="post", maxlen=250)

In [18]:
#Criando o modelo
model = keras.Sequential()
#Tendo 2 palavras com mesmo sentido como great day e good day, para nós, parece a mesma coisa, mas a rede neural irá pegar dois números diferentes para great e good, logo, o Embedding virá para determinar o significado de CADA palavra na sentença mapeando cada palavra em um espaço de vetor, ou seja, irá agrupar palavras com sentido similar
#Olhando as palavras que circundam as frases, ele irá criar vetores com diferentes angulos (em relação a x) para cada palavra, caso a rede neural perceba que ambas palavras possuem mesmo vizinhos (ou, consequentemente, mesmo significado), ele irá tentar apeoximar esses dois vetores
#O 10000 é o número de palavras (no caso, números) que irão entrar na Layer, e 16 é as 16 dimensões de saída que será utilizadas para representar os vetores criados
model.add(keras.layers.Embedding(10000,16))
#Como após o Embedding ele sai com muitas dimensões, o Globar Average realiza um scale down de dimensões para facilitar para a rede neural analisar tudo
model.add(keras.layers.GlobalAveragePooling1D())
#Essa layer são 16 neurônios todos conectados com o Global Average
model.add(keras.layers.Dense(16,activation="relu"))
#Esse é o Output, onde sairá um valor entre 0 e 1 que falará se o movie review é positivo ou negativo
model.add(keras.layers.Dense(1,activation="sigmoid"))

#Isto mostra o design da sua rede
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [19]:
#O binary cross entropy é uma cost function visionada para apenas 2 valores na saída,sendo representados por 0 ou 1
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [20]:
#For this specific model we will introduce a new idea of validation data. In the last tutorial when we trained the models accuracy after each epoch on the current training data, data the model had seen before. This can be problematic as it is highly possible the a model can simply memorize input data and its related output and the accuracy will affect how the model is modified as it trains. So to avoid this issue we will sperate our training data into two sections, training and validation. The model will use the validation data to check accuracy after learning from the training data. This will hopefully result in us avoiding a false confidence for our model.
x_val = train_data[:10000]
x_train = train_data[10000:]

y_val = train_labels[:10000]
y_train = train_labels[10000:]

In [21]:
#Batch_size --> será quantos reviews ele irá pegar simultâneamente
fitModel = model.fit(x_train,y_train, epochs=40, batch_size=512, validation_data=(x_val,y_val),verbose=1)

results = model.evaluate(test_data,test_labels)

print(results)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
15000/15000 [==============================] - 1s 87us/sample - loss: 0.6919 - accuracy: 0.5187 - val_loss: 0.6901 - val_accuracy: 0.5651
Epoch 2/40
15000/15000 [==============================] - 1s 57us/sample - loss: 0.6864 - accuracy: 0.6073 - val_loss: 0.6830 - val_accuracy: 0.6154
Epoch 3/40
15000/15000 [==============================] - 1s 64us/sample - loss: 0.6751 - accuracy: 0.6629 - val_loss: 0.6687 - val_accuracy: 0.7280
Epoch 4/40
15000/15000 [==============================] - 1s 81us/sample - loss: 0.6553 - accuracy: 0.7301 - val_loss: 0.6463 - val_accuracy: 0.7553
Epoch 5/40
15000/15000 [==============================] - 1s 95us/sample - loss: 0.6260 - accuracy: 0.7770 - val_loss: 0.6160 - val_accuracy: 0.7654
Epoch 6/40
15000/15000 [==============================] - 1s 83us/sample - loss: 0.5880 - accuracy: 0.7993 - val_loss: 0.5777 - val_accuracy: 0.7983
Epoch 7/40
15000/15000 [==============================] 

In [22]:
#Salvando o modelo criado:
model.save("model.h5")  # name it whatever you want but end with .h5

In [23]:
#Carregando o modelo:
model = keras.models.load_model("model.h5")

In [25]:
#Fazendo alguns predicts
test_review = test_data[0]
predict = model.predict([test_review])
f"Review: {decode_review(test_review)}"
f"Prediction: {str(predict[0])}"
f"Actual: {str(test_labels[0])}"

'Actual: 0'